In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
train = pd.read_csv("/kaggle/input/instant-gratification/train.csv")
test = pd.read_csv("/kaggle/input/instant-gratification/test.csv")
submission = pd.read_csv("/kaggle/input/instant-gratification/sample_submission.csv") 

# Stacking
여러가지 모델이 도출한 예측값을 보고 합산하여 최종 예측결과를 도출하는 방법

## Fold
사용하는 train dataset을 n개의 폴드로 나눠 n번의 학습을 진행하는 방법 (KFold)
n번의 학습에서 각각 모든 폴드는 validate dataset으로 진행되게 되며 validate dataset이 아닌 폴드는 모두 train dataset으로 사용됨

* OOF : Out Of Fold (모델이 학습, 테스트 예측한 결과)

In [ ]:
# 예시를 보이는 코드 (작동 X)
from sklearn.model_selection import KFold

oof_train = np.zeros((len(train), 1))
oof_test = np.zeros((len(train),1))
target = train["target"]
n = 3

# n_splits : fold 갯수, KFold는 해당 데이터를 적절히 나눠 인덱스를 출력해주는 툴
fold = KFold(n_splits=n, shuffle=True, random_state=42)

# train_ids, valid_ids는 배열로 주어짐
for train_ids, valid_ids in fold.split(train):
    x_train, y_train = train[train_id], target[train_id]
    x_valid, y_valid = train[valid_id], target[valid_id]
    model.fit(x_train, y_train)

    oof_train[valid_id] = model.predict(x_valid)
    oof_test += model.predict(test)

# test한 결과 n개가 더해져 있기 때문에 마지막에 나눠줌
oof_test = oof_test / n

## Meta Model (2단계 모델 만들기)
위에서 여러개의 모델이 도출한 결과인 OOF를 입력으로 가지는 2단계모델 (Stacking)을 만들기
보통 Gradient Boosting, Neural Network등을 사용함 2단계 모델은 파라미터를 너무 깊게 만들지 않음

In [ ]:
# 위 과정을 4번 진행하여 4개의 모델을 사용한다고 가정

# 여러 모델들의 oof를 이어 만들기
train_2nd_input = np.concatenate(
    (oof_train_model1, oof_train_model2, oof_train_model3, oof_train_model4), axis=1
)
test_2nd_input = np.concatenate(
    (oof_test_model1, oof_test_model2, oof_test_model3, oof_test_model4, axis=1)
)

oof_train_2nd = np.zeros((len(train_2nd_input), 1))
oof_test_2nd = np.zeros((len(test_2nd_input), 1))

n_2nd = 3
fold = KFold(n_splits=n_2nd, random_state=42)
for train_ids, valid_ids in fold.split(train_2nd_input):
    x_train, y_train = train_2nd_input[train_ids], target[train_ids]
    x_valid, y_valid = train_2nd_input[valid_ids], target[valid_ids]

    model.fit(x_train, y_train)
    oof_train_2nd[valid_ids] = model.predict(x_valid)
    oof_test_2nd += model.predict(test_2nd_input)
    
oof_test_2nd = oof_test_2nd / 3